In [105]:
from pandas import pandas as pd

In [106]:
from hiclass.metrics import precision, recall, f1

In [107]:
from sklearn.metrics import classification_report

In [108]:
from sklearn.preprocessing import MultiLabelBinarizer

In [109]:
import os

In [110]:
import ast

In [111]:
from utils.data import load 
from utils.dir import create_dir

In [112]:
def __load_json__(path):
    with open(path, 'r') as f:
        tmp = json.loads(f.read())

    return tmp

In [113]:

args = pd.Series({
    "root_dir": "/mnt/disks/data/",
    "dataset_path": "/mnt/disks/data/fma/fma_large", 
    "metadata_path": "/mnt/disks/data/fma/fma_metadata", 
    "embeddings": "music_style",
    "sequence_size": 1280,
    "train_id": "hierarchical_hiclass"
})



In [114]:

job_path = "/mnt/disks/data/fma/trains"

# In[15]:


train_path = os.path.join(job_path, args.train_id)

# In[16]:


base_path = os.path.join(args.root_dir, "fma")

# In[17]:


models_path = os.path.join(args.root_dir, "models")


# In[18]:


metadata_file = os.path.join(train_path, "metadata.json")
labels_file = os.path.join(train_path, "labels.json")
hiclass_effnet_path = os.path.join(train_path, 'hiclass_effnet')
hiclass_baseline_path = os.path.join(train_path, 'hiclass_baseline')

In [115]:
hiclass_effnet_path

'/mnt/disks/data/fma/trains/hierarchical_hiclass/hiclass_effnet'

In [116]:
dirs = os.listdir(hiclass_effnet_path)

In [117]:
dirs

['MLP1']

In [118]:
df_predict = pd.read_csv(os.path.join(hiclass_effnet_path,dirs[0],'predict.csv'))

In [119]:
def norm_labels(label):
    label = ast.literal_eval(label)
    #label = [[elemento for elemento in sublist if elemento != 0] for sublist in label]
    return label

In [120]:
df_predict['full_genre_id'] = df_predict.full_genre_id.apply(lambda x: norm_labels(x))

In [121]:
df_predict['y_pred'] = df_predict.y_pred.apply(lambda x: norm_labels(x))

In [122]:
y_true = df_predict.full_genre_id.values
y_pred = df_predict.y_pred.values

In [123]:
import numpy as np

In [124]:
#y_pred = [[sublista for sublista in lista if any(element != '' for element in sublista)] for lista in y_pred]

In [125]:
y_pred = [[[str(num) if num != '' else '' for num in sublist] for sublist in lista] for lista in y_pred]

In [126]:
#y_pred = [y[0] for y in y_pred]

In [127]:
y_true = [[[str(num) if num != '' else '' for num in sublist] for sublist in lista] for lista in y_true]

In [128]:
# Completar cada sublista com listas vazias até o comprimento máximo de 8
y_true_completado = [sublista + [[]] * (8 - len(sublista)) for sublista in y_true]

# Preencher cada lista vazia com strings vazias ('') até o tamanho de 5
for sublista in y_true_completado:
    for lista_vazia in sublista:
        lista_vazia.extend([''] * (5 - len(lista_vazia)))

In [129]:
y_true = y_true_completado

In [130]:
#y_true = np.array([[[elemento for elemento in sublista if elemento != ''] for sublista in lista] for lista in y_true])

In [131]:
len(y_true)

22228

In [132]:
len(y_pred)

22228

In [133]:
# Calculando a precisão usando a função precision do HiClass
precision_macro = precision(y_true, y_pred, average='macro')
recall_macro = precision(y_true, y_pred, average='macro')

precision_micro = precision(y_true, y_pred, average='micro')
recall_micro = precision(y_true, y_pred, average='micro')

f1_score_micro = 2 * precision_micro * recall_micro / (precision_micro + recall_micro)

#report = classification_report(y_true_flat, y_pred_flat)
## Imprimindo os resultados
print("Precisão (média micro):", precision_micro)
print("Precisão (média macro):", precision_macro)

print("Recall (média micro):", recall_micro)
print("Recall (média macro):", recall_macro)

print("F1 (média micro):", f1_score_micro)

Precisão (média micro): 0.407190036221357
Precisão (média macro): 0.4110537331722302
Recall (média micro): 0.407190036221357
Recall (média macro): 0.4110537331722302
F1 (média micro): 0.407190036221357


In [134]:
len(y_pred)

22228

In [157]:
y_true[0][0][1]

'58'

In [66]:
y_pred[0][0][1]

25

In [283]:
len(y_true[0][0])

5

In [284]:
# Calcular métricas para cada rótulo individualmente
y_t_all = []
y_p_all = []
for i in range(len(y_true[0][0])):
    print(f"Label {i}:")
    enc = MultiLabelBinarizer()
    y_t_level = []
    y_p_level = []
    for y_t,y_p in zip(y_true,y_pred):
        y_true_label = [sublist[i] for sublist in y_t]
        y_pred_label = [sublist[i] for sublist in y_p]
        y_t_level.append(y_true_label)
        y_p_level.append(y_pred_label)
    y_t_all.append(enc.fit_transform(y_t_level))
    y_p_all.append(enc.transform(y_p_level))
    
#print(recall(y_true_label, y_pred_label))

Label 0:
Label 1:
Label 2:


/home/bruno/.cache/pypoetry/virtualenvs/fma-hiclass-YXxd6Unq-py3.11/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) [''] will be ignored
  warnings.warn(


Label 3:
Label 4:


In [285]:
len(y_t_all[0][0])

16

In [286]:
len(y_p_all[0][0])

16

In [287]:
print(classification_report(y_t_all[0], y_p_all[0]))

              precision    recall  f1-score   support

           0       0.15      0.00      0.01      3044
           1       0.31      0.26      0.28      7122
           2       0.15      0.01      0.01      3285
           3       0.00      0.00      0.00       201
           4       0.00      0.00      0.00       377
           5       0.33      0.30      0.31      7404
           6       0.11      0.03      0.05      2804
           7       0.12      0.00      0.01      1280
           8       0.00      0.00      0.00       453
           9       0.11      0.01      0.01      1816
          10       0.00      0.00      0.00       420
          11       0.36      0.37      0.37      8207
          12       0.00      0.00      0.00       973
          13       0.04      0.00      0.01       934
          14       0.00      0.00      0.00       201
          15       0.17      0.00      0.01       468

   micro avg       0.32      0.18      0.24     38989
   macro avg       0.12   

/home/bruno/.cache/pypoetry/virtualenvs/fma-hiclass-YXxd6Unq-py3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [270]:
enc.classes

In [35]:
import hiclass
from hiclass import metrics
import numpy as np

# Suponha que você tenha rótulos verdadeiros (y_true) e rótulos previstos (y_pred)
Y_true = np.array([
    [["Retriever", "Golden Retriever"], ["Hound", "Dachshund"]],
    [["Retriever", "Labrador"]],
    [["Hound", "Dachshund"], ["Hound", "Beagle"]],
], dtype=object)

Y_pred = np.array([
    [["Retriever", "Golden Retriever"], ["Hound", "Dachshund"]],
    [["Retriever", "Labrador"]],
    [["Hound", "Dachshund"], ["Hound", "Beagle"]],
], dtype=object)

# Convertendo para o formato aceito pelo HiClass
y_true_flat = [[subitem for sublist in item for subitem in sublist] for item in Y_true]
y_pred_flat = [[subitem for sublist in item for subitem in sublist] for item in Y_pred]

# Calculando a precisão usando a função precision do HiClass
precision_micro = metrics.precision(y_true_flat, y_pred_flat, average='micro')
precision_macro = metrics.precision(y_true_flat, y_pred_flat, average='macro')

# Imprimindo os resultados
print("Precisão (média micro):", precision_micro)
print("Precisão (média macro):", precision_macro)

Precisão (média micro): 1.0
Precisão (média macro): 1.0
